In [351]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/c/titanic/train.csv
/kaggle/input/c/titanic/test.csv
/kaggle/input/c/titanic/gender_submission.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv


In [352]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier 

import warnings
warnings.filterwarnings('ignore')

In [353]:
%matplotlib inline

In [354]:
titanic = pd.read_csv('/kaggle/input/c/titanic/train.csv', header = 0, dtype={'Age': np.float64})
titanic.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [355]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [356]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [357]:
titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [358]:
#Name column can never decide survival of a person, hence we can safely delete it
del titanic["Ticket"]
del titanic["Fare"]
del titanic['Cabin']
del titanic["Name"]
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,1,0,3,male,22.0,1,0,S
1,2,1,1,female,38.0,1,0,C
2,3,1,3,female,26.0,0,0,S
3,4,1,1,female,35.0,1,0,S
4,5,0,3,male,35.0,0,0,S


In [359]:
print(titanic.select_dtypes(include=['category']))

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[891 rows x 0 columns]


In [360]:
titanic.select_dtypes(include=['object'])

,Sex,Embarked
0,male,S
1,female,C
2,female,S
3,female,S
4,male,S
...,...,...
886,male,S
887,female,S
888,female,S
889,male,C


In [361]:
list_str_obj_cols = titanic.columns[titanic.dtypes == "object"].tolist()
for str_obj_col in list_str_obj_cols:
    titanic[str_obj_col] = titanic[str_obj_col].astype("category")

titanic.dtypes

PassengerId       int64
Survived          int64
Pclass            int64
Sex            category
Age             float64
SibSp             int64
Parch             int64
Embarked       category
dtype: object

In [362]:
list_cat=titanic.columns[titanic.dtypes == "category"].tolist()
for str_c in list_cat:
    print(titanic[str_c].unique())


['male', 'female']
Categories (2, object): ['female', 'male']
['S', 'C', 'Q', NaN]
Categories (3, object): ['C', 'Q', 'S']


In [363]:
list_cat=titanic.columns[titanic.dtypes == "category"].tolist()
for str_c in list_cat:
    print(titanic[str_c].value_counts())


Sex
male      577
female    314
Name: count, dtype: int64
Embarked
S    644
C    168
Q     77
Name: count, dtype: int64


In [364]:
list_cat=titanic.columns[titanic.dtypes == "category"].tolist()
one_hot_encoded_data = pd.get_dummies(titanic, columns = list_cat) 
print(one_hot_encoded_data)

     PassengerId  Survived  Pclass   Age  SibSp  Parch  Sex_female  Sex_male  \
0              1         0       3  22.0      1      0       False      True   
1              2         1       1  38.0      1      0        True     False   
2              3         1       3  26.0      0      0        True     False   
3              4         1       1  35.0      1      0        True     False   
4              5         0       3  35.0      0      0       False      True   
..           ...       ...     ...   ...    ...    ...         ...       ...   
886          887         0       2  27.0      0      0       False      True   
887          888         1       1  19.0      0      0        True     False   
888          889         0       3   NaN      1      2        True     False   
889          890         1       1  26.0      0      0       False      True   
890          891         0       3  32.0      0      0       False      True   

     Embarked_C  Embarked_Q  Embarked_S

In [365]:
meanS= one_hot_encoded_data.Age.mean()
meanS

29.69911764705882

In [366]:
one_hot_encoded_data["Age"]=np.where(pd.isnull(one_hot_encoded_data.Age),meanS, one_hot_encoded_data["Age"])
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,1,0,3,male,22.0,1,0,S
1,2,1,1,female,38.0,1,0,C
2,3,1,3,female,26.0,0,0,S
3,4,1,1,female,35.0,1,0,S
4,5,0,3,male,35.0,0,0,S


In [367]:
one_hot_encoded_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [368]:
# one_hot_encoded_data.drop(columns=['Age'])

In [369]:
one_hot_encoded_data.replace({False: 0, True: 1}, inplace=True)

In [370]:
one_hot_encoded_data

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.000000,1,0,0,1,0,0,1
1,2,1,1,38.000000,1,0,1,0,1,0,0
2,3,1,3,26.000000,0,0,1,0,0,0,1
3,4,1,1,35.000000,1,0,1,0,0,0,1
4,5,0,3,35.000000,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.000000,0,0,0,1,0,0,1
887,888,1,1,19.000000,0,0,1,0,0,0,1
888,889,0,3,29.699118,1,2,1,0,0,0,1
889,890,1,1,26.000000,0,0,0,1,1,0,0


In [371]:
# meanNS=one_hot_encoded_data.Age.mean()
# meanNS

In [372]:
# one_hot_encoded_data.Age.fillna(meanNS,inplace=True)
# one_hot_encoded_data.head()

In [373]:
one_hot_encoded_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [374]:
list_cat=titanic.columns
list_cat

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Embarked'],
      dtype='object')

In [375]:
# del one_hot_encoded_data['Age']
# one_hot_encoded_data.head()

In [376]:
# survivedQ = one_hot_encoded_data[one_hot_encoded_data.Embarked == 'Q'][one_hot_encoded_data.Survived == 1].shape[0]
# survivedC = one_hot_encoded_data[one_hot_encoded_data.Embarked == 'C'][one_hot_encoded_data.Survived == 1].shape[0]
# survivedS = one_hot_encoded_data[one_hot_encoded_data.Embarked == 'S'][one_hot_encoded_data.Survived == 1].shape[0]
# print(survivedQ)
# print(survivedC)
# print(survivedS)

In [377]:
# survivedQ = one_hot_encoded_data[one_hot_encoded_data.Embarked == 'Q'][one_hot_encoded_data.Survived == 0].shape[0]
# survivedC = one_hot_encoded_data[one_hot_encoded_data.Embarked == 'C'][one_hot_encoded_data.Survived == 0].shape[0]
# survivedS = one_hot_encoded_data[one_hot_encoded_data.Embarked == 'S'][one_hot_encoded_data.Survived == 0].shape[0]
# print(survivedQ)
# print(survivedC)
# print(survivedS)

In [378]:
one_hot_encoded_data.isnull().sum().count()

11

In [379]:
one_hot_encoded_data.dropna(inplace=True)
one_hot_encoded_data.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,0,1,0,0,1
1,2,1,1,38.0,1,0,1,0,1,0,0
2,3,1,3,26.0,0,0,1,0,0,0,1
3,4,1,1,35.0,1,0,1,0,0,0,1
4,5,0,3,35.0,0,0,0,1,0,0,1


In [380]:
one_hot_encoded_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [381]:
one_hot_encoded_data.isnull().sum().count()

11

In [382]:
#Renaming "age" and "gender" columns
one_hot_encoded_data.rename(columns={'age':'Age'}, inplace=True)
one_hot_encoded_data.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,0,1,0,0,1
1,2,1,1,38.0,1,0,1,0,1,0,0
2,3,1,3,26.0,0,0,1,0,0,0,1
3,4,1,1,35.0,1,0,1,0,0,0,1
4,5,0,3,35.0,0,0,0,1,0,0,1


In [383]:
h=one_hot_encoded_data.columns[one_hot_encoded_data.isnull().any()]
h

Index([], dtype='object')

In [384]:
del one_hot_encoded_data['Sex_female']
del one_hot_encoded_data['Embarked_C']

In [385]:
one_hot_encoded_data

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Sex_male,Embarked_Q,Embarked_S
0,1,0,3,22.000000,1,0,1,0,1
1,2,1,1,38.000000,1,0,0,0,0
2,3,1,3,26.000000,0,0,0,0,1
3,4,1,1,35.000000,1,0,0,0,1
4,5,0,3,35.000000,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.000000,0,0,1,0,1
887,888,1,1,19.000000,0,0,0,0,1
888,889,0,3,29.699118,1,2,0,0,1
889,890,1,1,26.000000,0,0,1,0,0


In [386]:
# one_hot_encoded_data.rename(columns={"Sex_male": "Gender"})

In [387]:
x=one_hot_encoded_data.drop(columns=['Survived'])
y=one_hot_encoded_data['Survived']

In [388]:
one_hot_encoded_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Sex_male     891 non-null    int64  
 7   Embarked_Q   891 non-null    int64  
 8   Embarked_S   891 non-null    int64  
dtypes: float64(1), int64(8)
memory usage: 62.8 KB


In [389]:
titanic = pd.read_csv('/kaggle/input/c/titanic/test.csv', header = 0, dtype={'Age': np.float64})
titanic.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [390]:
titanic.describe()


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [391]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [392]:
#Name column can never decide survival of a person, hence we can safely delete it
del titanic["Ticket"]
del titanic["Fare"]
del titanic['Cabin']
del titanic["Name"]
titanic.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Embarked
0,892,3,male,34.5,0,0,Q
1,893,3,female,47.0,1,0,S
2,894,2,male,62.0,0,0,Q
3,895,3,male,27.0,0,0,S
4,896,3,female,22.0,1,1,S


In [393]:
list_str_obj_cols = titanic.columns[titanic.dtypes == "object"].tolist()
for str_obj_col in list_str_obj_cols:
    titanic[str_obj_col] = titanic[str_obj_col].astype("category")

titanic.dtypes

PassengerId       int64
Pclass            int64
Sex            category
Age             float64
SibSp             int64
Parch             int64
Embarked       category
dtype: object

In [394]:
list_cat=titanic.columns[titanic.dtypes == "category"].tolist()
one_hot_encoded = pd.get_dummies(titanic, columns = list_cat) 
print(one_hot_encoded)

     PassengerId  Pclass   Age  SibSp  Parch  Sex_female  Sex_male  \
0            892       3  34.5      0      0       False      True   
1            893       3  47.0      1      0        True     False   
2            894       2  62.0      0      0       False      True   
3            895       3  27.0      0      0       False      True   
4            896       3  22.0      1      1        True     False   
..           ...     ...   ...    ...    ...         ...       ...   
413         1305       3   NaN      0      0       False      True   
414         1306       1  39.0      0      0        True     False   
415         1307       3  38.5      0      0       False      True   
416         1308       3   NaN      0      0       False      True   
417         1309       3   NaN      1      1       False      True   

     Embarked_C  Embarked_Q  Embarked_S  
0         False        True       False  
1         False       False        True  
2         False        True      

In [395]:
meanS= one_hot_encoded.Age.mean()
meanS

30.272590361445783

In [396]:
one_hot_encoded["Age"]=np.where(pd.isnull(one_hot_encoded.Age),meanS, one_hot_encoded["Age"])
titanic.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Embarked
0,892,3,male,34.5,0,0,Q
1,893,3,female,47.0,1,0,S
2,894,2,male,62.0,0,0,Q
3,895,3,male,27.0,0,0,S
4,896,3,female,22.0,1,1,S


In [397]:
one_hot_encoded.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [398]:
del one_hot_encoded['Sex_female']
del one_hot_encoded['Embarked_C']

In [399]:
one_hot_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Age          418 non-null    float64
 3   SibSp        418 non-null    int64  
 4   Parch        418 non-null    int64  
 5   Sex_male     418 non-null    bool   
 6   Embarked_Q   418 non-null    bool   
 7   Embarked_S   418 non-null    bool   
dtypes: bool(3), float64(1), int64(4)
memory usage: 17.7 KB


In [400]:
x=one_hot_encoded_data.drop(columns=['Survived'])
y=one_hot_encoded_data['Survived']

In [401]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [402]:
x_train,x_test,y_train,y_test

(     PassengerId  Pclass        Age  SibSp  Parch  Sex_male  Embarked_Q  \
 301          302       3  29.699118      2      0         1           1   
 309          310       1  30.000000      0      0         0           0   
 516          517       2  34.000000      0      0         0           0   
 120          121       2  21.000000      2      0         1           0   
 570          571       2  62.000000      0      0         1           0   
 ..           ...     ...        ...    ...    ...       ...         ...   
 715          716       3  19.000000      0      0         1           0   
 767          768       3  30.500000      0      0         0           1   
 72            73       2  21.000000      0      0         1           0   
 235          236       3  29.699118      0      0         0           0   
 37            38       3  21.000000      0      0         1           0   
 
      Embarked_S  
 301           0  
 309           0  
 516           1  
 120      

In [403]:
PipelineDict={0:'Logistic Regression',1:'Decision Tree',2:'Random Forest',3:'SVM'}


In [404]:
LogisticRegressionPipeline=Pipeline([('myscaler',StandardScaler()),('mypca',PCA(n_components=0.95)),('logistic_classifier',LogisticRegression())])

In [405]:
DecisionTreePipeline=Pipeline([('myscaler',StandardScaler()),('mypca',PCA(n_components=0.95)),('Decision_Tree',DecisionTreeClassifier())])

In [406]:
RandomForestPipeline=Pipeline([('myscaler',StandardScaler()),('mypca',PCA(n_components=0.95)),('Random_Forest',RandomForestClassifier())])

In [407]:
LinearSVCPipeline=Pipeline([('myscaler',StandardScaler()),('mypca',PCA(n_components=0.95)),('Linear_SVC',LinearSVC())])

In [408]:
mypipeline=[LogisticRegressionPipeline,DecisionTreePipeline,RandomForestPipeline,LinearSVCPipeline]

In [409]:
for model in mypipeline:
    model.fit(x_train,y_train)

In [410]:
accuracy=0.0
classifier=0
pipeline=""

In [411]:
for i,model in enumerate(mypipeline):
    print("{} Test Accuracy:{}".format(PipelineDict[i],model.score(x_test,y_test)))

Logistic Regression Test Accuracy:0.7821229050279329
Decision Tree Test Accuracy:0.6871508379888268
Random Forest Test Accuracy:0.7988826815642458
SVM Test Accuracy:0.7932960893854749


In [412]:
for i,model in enumerate(mypipeline):
    if model.score(x_test,y_test)>accuracy:
        accuracy=model.score(x_test,y_test)
        pipeline=model
        classifier=i
        

In [413]:
print('classifier with best accuracy:{}'.format(PipelineDict[classifier]))

classifier with best accuracy:Random Forest


In [414]:
J=PipelineDict[classifier]

In [415]:
J

'Random Forest'

In [416]:
classifier

2

In [417]:
model

Pipeline(steps=[('myscaler', StandardScaler()),
                ('mypca', PCA(n_components=0.95)),
                ('Linear_SVC', LinearSVC())])

In [418]:
test_ids=one_hot_encoded["PassengerId"]

In [419]:
sub=model.predict(one_hot_encoded)

In [420]:
df=pd.DataFrame({"PassengerId":test_ids.values,"Survived":sub})

In [422]:
df.to_csv("submission.csv",index=False)